In [ ]:
import keyring        # for loading api token
import urllib.request # for encoding URL parameters
import pandas as pd   # for handling data frames
import json           # for handling json
import os             # for outputting the absolute path of the file containing the data

# Authentication

Please read the Authentication section of the [README](README.ipynb) to set up your acess token for the following code.  <span style="color:red"> WARNING: the below code will load the access token you saved from the README.ipynb</span>

In [ ]:
token = keyring.get_password("system", "canvas_token");
print("Loaded token with %d characters." % (len(token)))

# Configuration

The following configuration does not frequently change. Input your course ID found in the url of your canvas course.

In [ ]:
course_id = int(input("Enter your course id here: "))

This next one can change based on your rubric.  This can be found from the data returned from the assignments notebook or by looking at the outcomes section of canvas.

In [ ]:
rubric_id = int(input("Enter your rubric id here: "))

In [ ]:
config = {
        'base_url': 'https://usfca.instructure.com',
        'course_id': course_id,
        'rubric': rubric_id
}

# Request Setup

The following sets up the REST API request to get a rubric:

<https://canvas.instructure.com/doc/api/rubrics.html#method.rubrics_api.show>

In [ ]:
api_format = '{base_url}/api/v1/courses/{course_id}/rubrics/{rubric}'
api_url = api_format.format(**config)
print("URL:", api_url)

In [ ]:
params = {
    'per_page': 200,
    'include[]': 'assessments',
    'style':'full'
}

encoded = urllib.parse.urlencode(params)
print("Params:", encoded) # do not output api key 

In [ ]:
rest_call = '{}?access_token={}&{}'.format(api_url, token, encoded)
print("REST call is %d characters." % (len(rest_call)))

# Fetch Data

Fetch the JSON data from the API.

In [ ]:
# due to the format of this json object, some preprocessing using  the json library needs to be done then converted to a dataframe
with urllib.request.urlopen(rest_call) as url:
    data = json.loads(url.read().decode())  
data = pd.DataFrame.from_dict(data, orient='index')
print('Loaded {} rows and {} columns.'.format(*data.shape))
# The column should be the rubric 
# the rows are the information about the rubric

In [ ]:
# output columns (should be one due to requesting one rubric)
print('Columns:', list(data.columns.values))

In [ ]:
data.head() # look at the results

# Obtain Assessments

Attempting to obtain assessments information if possible in the rubric.  These assessments are the feeedbacks students gave each other in the submission.

In [ ]:
try:
    assessments = data.loc['assessments']
except KeyError:
    print("There are no assessments here please input a new assignment or check that your access token is accurate")

In [ ]:
converted = pd.DataFrame.from_dict(assessments) # we only need the assessments from the data gathered from the api

In [ ]:
converted

# Create Assessments Dataframe

Creating dataframe out of the assessments

In [ ]:
expanded = converted.explode("assessments") # converted is a single row containing a list of all assessments of a rubric
# we want to convert that list to many rows per feedback
expanded.reset_index(inplace=True, drop=True)

In [ ]:
expanded

In [ ]:
expanded.loc[0, "assessments"] # testing lokking at a single feedback

# Wrangling

Some of the columns could use some wrangling.

In [ ]:
wrangled = expanded.copy()

In [ ]:
# convert the dictionary that corresponds to the feedback into a row of a dataframe for all rows
assessments = wrangled.assessments.apply(pd.Series) 

In [ ]:
assessments.head() # see how output has changed

In [ ]:
# rubric association has more information in the form of a dictionary so do the same as above
rubric_association = assessments.rubric_association.apply(pd.Series) 

In [ ]:
# renaming columns from rubric association since some have the same names as columns from the assessments
rubric_association.columns = ["rubric_association_" + item for item in rubric_association.columns.tolist()] 

In [ ]:
rubric_association.head() # make sure columns are renamed

In [ ]:
reviews = pd.concat([assessments, rubric_association], axis=1) # combine the riginal assessments with the rubric associations

In [ ]:
reviews

In [ ]:
reviews = reviews.loc[:,~reviews.columns.duplicated()] # removing any duplicated columns

In [ ]:
reviews = reviews.explode("data") # the data field corresponds to each question in the feedback

In [ ]:
reviews.head()

In [ ]:
# as done above, the data is in the format of a dictionary and can have the same column names
reviews_data = reviews.data.apply(pd.Series)
reviews_data.columns = ["reviews_data_" + item for item in reviews_data.columns.tolist()] 

In [ ]:
reviews_data.head() # make sure columns are renamed properly

In [ ]:
final = pd.concat([reviews_data, reviews], axis=1) # combine the data again

In [ ]:
final.reset_index(inplace=True)

In [ ]:
final.head() # done combining data

In [ ]:
final.drop(columns=["rubric_association", "data"], inplace=True) # since both columns were expanded we can safely drop them

# Optional

Do this if you already saved a peer review to a file and want to add another to it from another assignment

In [ ]:
# peer_reviews = pd.read_csv('peer_reviews.csv')
# final = pd.concat([final, peer_reviews], axis=0, sort=False)

# Output Results

Output the results to CSV.

In [ ]:
path = 'peer_reviews.csv'

In [ ]:
final.to_csv(path, header = True, index = False)

In [ ]:
os.path.abspath('peer_reviews.csv') # the absolute path of the result